In [1]:
#Import Modules
import numpy as np
import pandas as pd
import scipy
import shap
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc

c:\Users\grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load Data
admission_annotation = pd.read_csv(f"../Key data files/admission_annotation.csv").rename(columns = {'X':'patient_ids'})
admission_norm_gene_exp = pd.read_csv(f"../Key data files/admission_norm_gene_exp_df.csv").rename(columns = {'Unnamed: 0':'gene_ids'})
delirium_cohort_demographics = pd.read_excel(f"../Key data files\delirium cohort demographics.xlsx").iloc[:128,:]
gene_symbols = pd.read_csv(f"../Key data files\gene_symbols.csv").iloc[:,1:]
serial_norm_gene_exp = pd.read_csv(f"../Key data files\serial_norm_gene_exp_df.csv").rename(columns = {'Unnamed: 0':'gene_symbols'})
serial_samples_annotation = pd.read_csv(f"../Key data files\serial_samples_annotation.csv").rename(columns = {'X':'patient_ids'})

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\grace\AppData\Local\Temp\ipykernel_7092\715775129.py:4: SyntaxWarning: invalid escape sequence '\d'
  delirium_cohort_demographics = pd.read_excel(f"../Key data files\delirium cohort demographics.xlsx").iloc[:128,:]
C:\Users\grace\AppData\Local\Temp\ipykernel_7092\715775129.py:5: SyntaxWarning: invalid escape sequence '\g'
  gene_symbols = pd.read_csv(f"../Key data files\gene_symbols.csv").iloc[:,1:]
C:\Users\grace\AppData\Local\Temp\ipykernel_7092\715775129.py:6: SyntaxWarning: invalid escape sequence '\s'
  serial_norm_gene_exp = pd.read_csv(f"../Key data files\serial_norm_gene_exp_

In [3]:
s = serial_norm_gene_exp.set_index('gene_symbols').T.reset_index()

In [4]:
d7 = serial_samples_annotation[serial_samples_annotation['Day'] == 7].reset_index(drop= True)["Diagnosis"]
d4 = serial_samples_annotation[serial_samples_annotation['Day'] == 4].iloc[:-1,:].reset_index(drop= True)["Diagnosis"]

In [5]:
day_7 = s[serial_samples_annotation['Day'] == 7].reset_index(drop= True).iloc[:,1:]
day_4 = s[serial_samples_annotation['Day'] == 4].iloc[:-1,:].reset_index(drop= True).iloc[:,1:]

day_7["Delirium"] = d7
day_4["Delirium"] = d4

day_4_d = day_4[day_4["Delirium"] == 0]
day_4_nd = day_4[day_4["Delirium"] == 1]
day_7_d = day_7[day_7["Delirium"] == 0]
day_7_nd = day_7[day_7["Delirium"] == 1]

In [6]:
from scipy.stats import ttest_ind
p_values1 = []
p_values2 = []

for i in range(day_4.shape[1]):
    t_statistic, p_value = ttest_ind(day_4_d.iloc[:,i], day_7_d.iloc[:,1])
    p_values1.append(p_value)
    t_statistic, p_value = ttest_ind(day_4_nd.iloc[:,i], day_7_nd.iloc[:,1])
    p_values2.append(p_value)


c:\Users\grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [7]:
df = admission_norm_gene_exp.set_index('gene_ids')
df.shape

(12683, 112)

In [11]:
sets = serial_norm_gene_exp.set_index('gene_symbols')[(diff > 0.5).tolist()[1:]].index

In [12]:
ids = gene_symbols.set_index('gene_symbols').T[sets].T['gene_ids'].tolist()
ids

t = admission_norm_gene_exp['gene_ids'].tolist()

intersection = list(set(ids).intersection(set(t)))

In [13]:
admission_norm_gene_exp.set_index('gene_ids').T[intersection].to_csv('serial features.csv')

In [26]:
a = admission_norm_gene_exp.set_index('gene_ids').T[intersection].T.index.to_frame().reset_index(drop = True)

In [31]:
a.to_excel("serial.xlsx")


In [29]:
gene_symbols.merge(a, on = "gene_ids").to_excel("serial.xlsx")

In [10]:
diff = np.array(p_values1) - np.array(p_values2)